# AKS Cookbook

## 🧪 Prompt Flow on AKS

![visual](visual.png)

Playground to try the [Azure AI Studio Prompt Flow](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/prompt-flow) with AKS.  
Prompt flow is a development tool designed to streamline the entire development cycle of AI applications powered by Large Language Models (LLMs). Prompt flow provides a comprehensive solution that simplifies the process of prototyping, experimenting, iterating, and deploying your AI applications.  
Prompt flow is available independently as an open-source project on [GitHub](https://github.com/microsoft/promptflow).

▶️ Click on the `Run All` button to execute all the subsequent steps in sequence, or run each step individually by executing the cells one at a time.

### TOC
- [📖 Prerequisites](#prerequisites)
- [0️⃣ Initialize notebook variables](#0)
- [1️⃣ Create the Azure Resource Group](#1)
- [2️⃣ Create deployment using 🦾 Bicep](#2)
- [3️⃣ Get the deployment outputs](#3)
- [4️⃣ Connect to the AKS cluster](#4)
- [5️⃣ Enable Prompt Flow tracing (optional)](#5)
- [6️⃣ Update the connection with APIM Gateway URL and APIM Subscription Key](#6)
- [🧪 Test the Flow locally](#localtest)
- [7️⃣ Create the Flow in the AI Studio Project (optional)](#7)
- [8️⃣ Build the Flow in the Docker format](#8)
- [9️⃣ Deploy the Flow to an Azure Container App](#9)
- [🧪 Test the Flow through APIM](#requests)
- [🔍 Analyze Application Insights requests](#portal)
- [🗑️ Clean up resources](#clean)



<a id='prerequisites'></a>
### 📖 Prerequisites
Install the following prerequisites for this demo

In [ ]:
import os, utils
output = utils.run(f"pip install keyrings.alt promptflow promptflow-tools promptflow-azure")

<a id='0'></a>
### 0️⃣ Initialize notebook variables
You can use this notebook with existing resources or to create the necessary resources.  
Adjust the location parameters according your preferences and on the [product availability by Azure region](https://azure.microsoft.com/en-us/explore/global-infrastructure/products-by-region/?cdn=disable).

In [ ]:
import os, time, json, requests, utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming convention
resource_group_location = "eastus2"
aks_prefix_name = "aks"
aks_nodepool_count = 1
aks_nodepool_vm_size = 'standard_d2s_v3'
acr_prefix_name = 'acr'

openai_prefix_name = "openai"
openai_resources_sku = "S0"
openai_model_name = "gpt-35-turbo"
openai_model_version = "0125"
openai_deployment_name = "gpt-35-turbo"

log_analytics_prefix_name = "workspace"
app_insights_prefix_name = 'insights'

ai_studio_hub_prefix_name = 'hub'
ai_studio_project_name = 'project'
storage_account_prefix_name = 'storage'
keyvault_prefix_name = 'keyvault'

flow_name = 'basic-chat' # the local folder must have the same name

utils.print_ok('Notebook initialized')

<a id='1'></a>
### 1️⃣ Verify the Azure CLI and connected Azure subscription
The following commands ensure that you have the latest version of the Azure CLI and relevant extensions installed while also verifying that the Azure CLI is connected to your Azure subscription.

In [ ]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")
if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    subscription_id = output.json_data['id']
    tenant_id = output.json_data['tenantId']
output = utils.run("az provider register --namespace Microsoft.ContainerService --wait", "Microsoft.ContainerService registered in your subscription", "Failed to register Microsoft.ContainerService")
output = utils.run("az provider register --namespace Microsoft.KubernetesConfiguration --wait", "Microsoft.KubernetesConfiguration registered in your subscription", "Failed to register Microsoft.KubernetesConfiguration")
output = utils.run("az extension add --name k8s-extension", "az k8s-extension installed", "Failed to install az k8s-extension")
output = utils.run("az extension update --name k8s-extension", "az k8s-extension updated", "Failed to update az k8s-extension")
output = utils.run("az extension add --name aks-preview", "az aks-preview extension installed", "Failed to install az aks-preview extension")
output = utils.run("az extension update --name aks-preview", "az aks-preview extension updated", "Failed to update az aks-preview extension")


<a id='2'></a>
### 2️⃣ Create deployment with BICEP 🦾
All resources deployed in this lab will be created within the designated resource group.   
The following step creates an AKS cluster using a BICEP deployment. 

In [ ]:
utils.create_resource_group(True, resource_group_name, resource_group_location)
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "aksPrefixName": { "value": aks_prefix_name },
        "nodePoolCount": { "value": aks_nodepool_count },
        "nodePoolVMSize": { "value": aks_nodepool_vm_size },
        "acrPrefixName": { "value": acr_prefix_name },
        "openAIPrefixName": { "value": openai_prefix_name },
        "openAIDeploymentName": { "value": openai_deployment_name },
        "openAISku": { "value": openai_resources_sku },
        "openAIModelName": { "value": openai_model_name },
        "openAIModelVersion": { "value": openai_model_version },
        "logAnalyticsPrefixName": { "value": log_analytics_prefix_name },
        "applicationInsightsPrefixName": { "value": app_insights_prefix_name },
        "aiStudioHubPrefixName": { "value": ai_studio_hub_prefix_name },
        "aiStudioProjectName": { "value": ai_studio_project_name },
        "storageAccountPrefixName": { "value": storage_account_prefix_name }, 
        "keyVaultPrefixName": { "value": keyvault_prefix_name }
    }
}    
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json", 
                f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")


<a id='3'></a>
### 3️⃣ Retrieve deployment outputs

In [ ]:
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")
if output.success and output.json_data:
    aks_resource_name = utils.get_deployment_output(output, 'aksResourceName', 'AKS Resource Name')
    acr_resource_name = utils.get_deployment_output(output, 'acrResourceName', 'ACR Resource Name')
    openai_api_endpoint = utils.get_deployment_output(output, 'openAIEndpoint', 'OpenAI API Endpoint')
    openai_api_key = utils.get_deployment_output(output, 'openAIKey')
    ml_project_name = utils.get_deployment_output(output, 'projectName', 'Project Name')
    ml_project_id = utils.get_deployment_output(output, 'projectId', 'Project Id')



<a id='4'></a>
### 4️⃣ Connect to the AKS cluster
Configure kubectl to connect to your Kubernetes cluster using the [az aks get-credentials](https://learn.microsoft.com/en-us/cli/azure/aks?view=azure-cli-latest#az-aks-get-credentials) command. This command downloads credentials and configures the Kubernetes CLI to use them.

In [ ]:
output = utils.run(f"az aks get-credentials --resource-group {resource_group_name} --name {aks_resource_name} --overwrite-existing",
             f"Credentials for AKS cluster '{aks_resource_name}' configured",
             f"Failed to configure credentials for AKS cluster '{aks_resource_name}'")


<a id='5'></a>
### 5️⃣ Enable Prompt Flow tracing (optional)

Prompt flow [tracing feature](https://microsoft.github.io/promptflow/how-to-guides/tracing/index.html) enables users to trace LLM calls, functions and even LLM frameworks. Besides, with promptflow[azure] installed, prompt flow can also log traces to an Azure ML workspace or Azure AI project, which makes it possible to share traces with your team members.

In [ ]:
! pf config set trace.destination=azureml:/{ml_project_id}

<a id='6'></a>
### 6️⃣ Update the connection 



In [ ]:
! pf connection create --file ./{flow_name}/azure_openai.yaml --set api_key={openai_api_key} api_base={openai_api_endpoint} --name open_ai_connection

<a id='localtest'></a>
### 🧪 Test the Flow locally


In [ ]:
! pf flow test --flow {flow_name} --inputs question="What's OpenAI?"

<a id='7'></a>
### 7️⃣  Create the Flow in the AI Studio Project (optional)

This feature is useful for sharing the flow with other users and leveraging the capabilities of Azure AI Studio. Open the [docs](https://microsoft.github.io/promptflow/cloud/azureai/manage-flows.html#create-a-flow) to learn more.

In [ ]:
! pfazure flow create --flow {flow_name} --set display_name={flow_name} type=chat description="Basic Chat Flow" -g {resource_group_name} -w {ml_project_name}


<a id='8'></a>
### 8️⃣ Build the Flow in the Docker format

This step will generate a build folder with a Dockerfile and prompt flow artifacts. Open the [docs](https://microsoft.github.io/promptflow/how-to-guides/deploy-a-flow/deploy-using-docker.html) to learn more.


In [ ]:
! pf flow build --source {flow_name} --output build --format docker

<a id='9'></a>
### 9️⃣ Deploy the Flow on AKS

In [ ]:
utils.run(f"az acr build --image {flow_name}:latest --resource-group {resource_group_name} --registry {acr_resource_name} --file build/Dockerfile build/. --no-logs", "Image was successfully built", "Failed to build the image")
utils.run(f"kubectl create configmap openai-config --from-literal=openai-key={openai_api_key}")

utils.run(f"kubectl apply -f {flow_name}/manifest.yaml")
utils.run(f"kubectl set image deployment {flow_name} {flow_name}={acr_resource_name}.azurecr.io/{flow_name}:latest")
utils.run(f"kubectl rollout restart deployment/{flow_name}")


<a id='requests'></a>
### 🧪 Test the Flow


In [ ]:
wait_time = 60 # in seconds
sleep_time = 5 # in seconds
request_timeout = 60 # in seconds
message = "Application is not ready yet. Check the logs."
while wait_time - sleep_time > 0:
    output = utils.run("kubectl get pods -l app=basic-chat -o jsonpath=\"{..status.phase}\"")
    if output.success and output.text == "Running":
        output = utils.run("kubectl get service basic-chat --output jsonpath={..status.loadBalancer.ingress[0]}")
        if output.success and output.json_data:
            ingress_ip = output.json_data['ip']
            utils.print_info(f"Service External IP: {ingress_ip}")
            try:
                response = requests.get(f"http://{ingress_ip}", timeout=request_timeout)
                if (response.status_code == 200):
                    utils.print_ok(f"Application ready @ http://{ingress_ip}")
                    message = None
                    break
            except:
                message = "The application is currently unresponsive..."
    wait_time -= sleep_time
    time.sleep(sleep_time)
if message:
    utils.print_warning(message)
else:    
    response = requests.post(f"http://{ingress_ip}/score", json = {"question": "Which is the biggest football club in Portugal?"})
    if (response.status_code == 200):
        data = json.loads(response.text)
        print("💬 ", data.get("answer"))
    else:
        print(response.text)


<a id='clean'></a>
### 🗑️ Clean up resources
When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered. Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.